<a href="https://colab.research.google.com/github/JHyunjun/torch_Generative-Adversarial-Network/blob/main/torch_PROJECT_TSGAN_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy import array
import pandas as pd
import numpy as np
import pandas_datareader.data as pdr
import matplotlib.pyplot as plt
import datetime
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, MinMaxScaler

#Data Load
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Batt/train_v.txt') 
#df_train = df_train.astype(int)
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Batt/test_v.txt')
train_data_input = df_train.iloc[:,:-1]
train_data_output = df_train.iloc[:,-1]
test_data_input = df_test.iloc[:,:-1]
test_data_output = df_test.iloc[:,-1]

train_data = df_train.iloc[:,:]
test_data = df_test.iloc[:,:]

trainSet_for_result = train_data
testSet_for_result = test_data

print("train data input : ",train_data_input.shape,train_data_input)
print("train data output : ",train_data_output.shape, train_data_output)
print("train_data : ",train_data.shape,train_data)
print("test_data : ",test_data.shape,test_data)

In [ ]:
pip install pytictoc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from pytictoc import TicToc

# TicToc 클래스 생성
t = TicToc()
t.tic()

time = 1800
#train_gan = df_train.iloc[0:len(df_train),-1]
train_gan = df_train.iloc[:time, -4:].values
print(train_gan.shape)

train_gan2 = df_test.iloc[:time, :]
#train_gan2 = train_gan2.to_numpy()
print(train_gan)

In [ ]:
'''
#Normalization
norm_max = np.max(train_gan)
norm_min = np.min(train_gan)        
#norm_max = 750
#norm_min = 600

def MinMaxScaler(data):
  #denom = np.max(data,0) - np.min(data,0)
  denom = norm_max - norm_min
  #nume = data-np.min(data,0)
  nume = data - norm_min
  return nume/denom

def back_MinMax(data,max,min):
  max = norm_max
  min = norm_min
  diff = max - min
  back = data * diff + min
  return back

#train_gan_norm = MinMaxScaler(train_gan)
print(train_gan.shape)
train_gan_max = np.max(train_gan)
train_gan_min = np.min(train_gan)
train_gan = MinMaxScaler(train_gan)
train_gan = torch.tensor(train_gan)
#train_gan_flatten = torch.flatten(train_gan)
#train_gan_flatten = train_gan_flatten.double().type()
train_gan = torch.tensor(train_gan, dtype = torch.float32)
train_gan_flatten = train_gan
#print(train_gan_flatten)
'''

In [ ]:
#Normalization

#print("Before Norm : ",train_data_input.shape, train_data_input)

def origin_minmax(data) : 
  arr_max = np.zeros(data.shape[1])
  arr_min = np.zeros(data.shape[1])
  print(arr_min)
  print(data.shape[1])
  for i in range(data.shape[1]) : 
    print(i)
    max = np.max(data[:,i])
    min = np.min(data[:,i]) 
    arr_max[i] = max
    arr_min[i] = min
    pass
  return arr_max, arr_min

arr_max1, arr_min1 = origin_minmax(train_gan)
#print(arr_max1, arr_min1)

def MinMaxScaler(data):
  for i in range(data.shape[1]) : 
    max = np.max(data[:,i])
    min = np.min(data[:,i])
    data[:,i] = (data[:,i] - min) / (max - min + 1e-7)
    pass
  return data

train_gan = MinMaxScaler(train_gan)

def back_MinMax(data,max,min):
  print(max)
  print(min)
  for i in range(data.shape[1]) : 
    data[:,i] = data[:,i] * (max[i] - min[i]) + min[i]
    pass
  return data

#print("After Norm : ",train_gan.shape, train_gan)
#print("After De_Norm : ",back_MinMax(train_gan, arr_max1, arr_min1))
train_gan = torch.tensor([train_gan], dtype = torch.float32)
print(train_gan.shape)

In [ ]:
'''
##train_gan_norm = MinMaxScaler(train_gan)
print(train_gan2.shape)
train_gan2_max = np.max(train_gan2)
train_gan2_min = np.min(train_gan2)
train_gan2 = MinMaxScaler(train_gan2)
train_gan2 = torch.tensor(train_gan2)
#train_gan_flatten = torch.flatten(train_gan)
#train_gan_flatten = train_gan_flatten.double().type()
train_gan2 = torch.tensor(train_gan2, dtype = torch.float32)
train_gan2_flatten = train_gan2
#print(train_gan2_flatten)
print("train_gan1 : " ,train_gan_max, train_gan_min)
print("train_gan2 : ",train_gan2_max, train_gan2_min)
'''

In [ ]:
train_gan = train_gan.transpose(1,2)
print(train_gan.shape) # [1,2,60]
Data_size = train_gan.shape[0]

#print(Data_size)

dim_size = 128 #conv1d filter size

class Discriminator(nn.Module):
    
    def __init__(self):
        # initialise parent pytorch class
        super().__init__()
        self.a = nn.Conv1d(train_gan.shape[1], dim_size, kernel_size = 5, padding = 2, padding_mode = 'zeros')
        self.b = nn.LeakyReLU(0.02)

        self.c = nn.Conv1d(dim_size, dim_size , kernel_size = 5, padding = 2, padding_mode = 'zeros')
        self.d = nn.LeakyReLU(0.02)

        self.e = nn.LayerNorm(train_gan.shape[2] * dim_size)

        self.f = nn.Linear(train_gan.shape[2] * dim_size, 1)
        self.g = nn.Sigmoid()

        # define neural network layers
        '''
        self.model = nn.Sequential(
            nn.Conv1d(2, 2, kernel_size = 5, padding = 2, padding_mode = 'zeros'),
            nn.LeakyReLU(0.02),
            nn.Conv1d(2, 1, kernel_size = 5, padding = 2, padding_mode = 'zeros'),
            nn.LeakyReLU(0.02),
            nn.Linear(60, 60), # (Datasize,latent_space)
            nn.LeakyReLU(0.02),
            nn.LayerNorm(60), 
            nn.Linear(60, 1), # (latent_space ,1)
            nn.Linear(1,1),
            nn.Sigmoid()
            
        )
        '''
        
        # create loss function
        self.loss_function = nn.BCELoss()

        # create optimiser, simple stochastic gradient descent
        self.optimiser = torch.optim.Adam(self.parameters(), lr=0.0001)

        # counter and accumulator for progress
        self.counter = 0;
        self.progress = []
        pass
    
    def forward(self, inputs):
        # simply run model
        x = self.a(inputs)
        #print("A : ",x.shape)
        x = self.b(x)
        #print("B : ",x.shape)
        x = self.c(x)
        #print("C : ",x.shape)
        x = self.d(x)
        #print("D : ",x.shape)
        x = torch.flatten(x)
        #print("flatten : ",x.shape)
        x = self.e(x)
        #print("E : ",x.shape)
        x = self.f(x)
        #print("F : ",x.shape)
        x = self.g(x)
        #print("G : ",x.shape)

        return x


    '''
    def forward(self, inputs):
        # simply run model
        print("in Discrimiatnor Forward : ",inputs.shape)
        return self.model(inputs)
    '''

    def train(self, inputs, targets):
        # calculate the output of the network
        outputs = self.forward(inputs)

        # calculate loss
        loss = self.loss_function(outputs, targets)

        # increase counter and accumulate error every 10
        self.counter += 1;
        if (self.counter % 10 == 0):
            self.progress.append(loss.item())
            pass
        if (self.counter % 1000 == 0):
            print("counter = ", self.counter)
            pass

        # zero gradients, perform a backward pass, update weights
        self.optimiser.zero_grad()
        loss.backward()
        self.optimiser.step()
        pass
    
    def plot_progress(self):
        df = pd.DataFrame(self.progress, columns=['Discriminator loss'])
        df.plot(ylim=(0), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0, 0.25, 0.5, 1.0, 5.0))
        pass
    
    pass

In [ ]:
def generate_random(size) :
  random_data = torch.rand(size)
  return random_data

def generate_random_seed(size):
    random_data = torch.randn(size)
    return random_data  

def generate_random_image(size) : 
  random_data = torch.rand(size)
  return random_data

In [ ]:
#Training the Discriminator
N = 10  #iterations
#train_gan_flatten = train_gan_flatten.double
#print((train_gan_flatten.double)) #pandas.core.frame.DataFrame

D = Discriminator()
targets = torch.FloatTensor([1.0])
non_target = torch.FloatTensor([0.0])

#for i in range(N) : 
#  D.train(train_gan_flatten, targets)
#  D.train(generate_random_image(Data_size), non_target)

#  pass

In [ ]:
#Checking the performance of Discriminator
#D.plot_progress()


class View(nn.Module):
    def __init__(self, shape):
        super().__init__()
        self.shape = shape,

    def forward(self, x):
        return x.view(*self.shape)


In [ ]:
# Constructing Generator

latent_space = 200

class Generator(nn.Module) : 
  def __init__(self) : 
    super().__init__()
    '''
    self.a = nn.Conv1d(1, dim_size, kernel_size = 5, padding = 2, padding_mode = 'zeros')
    self.b = nn.LeakyReLU(0.02)

    self.c = nn.Conv1d(dim_size, dim_size , kernel_size = 5, padding = 2, padding_mode = 'zeros')
    self.d = nn.LeakyReLU(0.02)

    self.e = nn.LayerNorm(latent_space * dim_size)

    #self.f = nn.Linear(latent_space * dim_size, latent_space * dim_size) #12800
    #self.f_1 = nn.LeakyReLU(0.02) #12800

    self.g_0 = nn.Linear(latent_space * dim_size, train_gan.shape[1] * train_gan.shape[2]) #2400
    self.g = nn.Sigmoid() #2400
    '''
    
    self.model = nn.Sequential(
        nn.Linear(latent_space,latent_space*4),
        nn.LeakyReLU(0.02),
        nn.LayerNorm(latent_space*4),

        nn.Linear(latent_space*4,latent_space*4),
        nn.LeakyReLU(0.02),
        nn.LayerNorm(latent_space * 4),

        nn.Linear(latent_space * 4, train_gan.shape[1] * train_gan.shape[2]),
        nn.Sigmoid(), #과연 Sigmoid가 최선인가

        View((1, train_gan.shape[1], train_gan.shape[2]))   
    )
    

    self.optimiser = torch.optim.Adam(self.parameters(), lr = 0.0001)
    self.counter = 0
    self.progress = []

    pass

  def forward(self, inputs) : 
    '''
    inputs = inputs.reshape(1,1,latent_space)
    x = self.a(inputs)
    x = self.b(x)
    x = self.c(x) 
    x = self.d(x)
    x = torch.flatten(x)
    x = self.e(x) 
    #x = self.f(x) 
    #x = self.f_1(x)
    x = self.g_0(x)
    x = self.g(x) #Sigmoid
    x = x.reshape(1, train_gan.shape[1] , train_gan.shape[2]) # [1,2,600]
    '''
    return self.model(inputs)

  def train(self, D, inputs, targets) : 
    g_output = self.forward(inputs)
    d_output = D.forward(g_output)
    #loss = D.forward(g_output)
    loss = D.loss_function(d_output, targets)

    self.counter+=1;
    if (self.counter % 10 == 0) :
      self.progress.append(loss.item())
      pass

    self.optimiser.zero_grad()
    loss.backward()
    self.optimiser.step()

  def plot_progress(self):
        df = pd.DataFrame(self.progress, columns=['Generator loss'])
        df.plot(ylim=(0), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0, 0.25, 0.5, 1.0, 5.0))
        
  pass

In [ ]:
#Training both Generator and Discriminator for pattern
D = Discriminator()
G = Generator()

epochs1 = 10000

#iteration1 = int(len(train_gan)/Data_size)*2 - 1
iteration1 = 1
print("Data size : ", Data_size, "len(train_gan) : ", len(train_gan), " iteration : ", iteration1)

for i in range(iteration1) : #i는 0부터 시작 
  print("First Iteration : ", i)
  for epoch in range(epochs1) :
    #ref = train_gan[(i) * Data_size : (i+1) * Data_size]
    #ref = train_gan[(i)*int(Data_size/2) : (i+2)*int(Data_size/2)] #Normalizing된 신호

    ref = train_gan
    #print("1 : " ,ref.shape) # [60,2]
    #print(ref)
    #ref = torch.tensor(ref, dtype = torch.float32)
    #ref = torch.flatten(ref) # [120]
    #print("2 : ", ref.shape) # [120]
    D.train(ref, torch.FloatTensor([1.0]))
    D.train(G.forward(generate_random_seed(latent_space)).detach(),torch.FloatTensor([0.0]))
    G.train(D, generate_random_seed(latent_space), torch.FloatTensor([1.0]))

    '''
    #ref2 = train_gan[len(train_gan)-((i+2)*int(Data_size/2)) : len(train_gan)-((i)*int(Data_size/2))] #Time-Inverse
    ref2 = train_gan[len(train_gan)-(i+1)*int(Data_size) : len(train_gan)-(i)*int(Data_size)] #Time-Inverse
    ref2 = torch.tensor(ref2, dtype = torch.float32)  
    D.train(ref2, torch.FloatTensor([1.0]))
    D.train(G.forward(generate_random_seed(latent_space)).detach(),torch.FloatTensor([0.0]))
    G.train(D, generate_random_seed(latent_space), torch.FloatTensor([1.0]))

    
    #ref3 = train_gan2[(i)*int(Data_size/2) : (i+2)*int(Data_size/2)] #Normalizing된 신호
    ref3 = train_gan2[(i) * Data_size : (i+1) * Data_size]
    ref3 = torch.tensor(ref3, dtype = torch.float32)
    D.train(ref3, torch.FloatTensor([1.0]))
    D.train(G.forward(generate_random_seed(latent_space)).detach(),torch.FloatTensor([0.0]))
    G.train(D, generate_random_seed(latent_space), torch.FloatTensor([1.0]))

    #ref4 = train_gan2[len(train_gan)-((i+2)*int(Data_size/2)) : len(train_gan)-((i)*int(Data_size/2))] #Time-Inverse
    ref4 = train_gan2[len(train_gan)-(i+1)*int(Data_size) : len(train_gan)-(i)*int(Data_size)]
    ref4 = torch.tensor(ref4, dtype = torch.float32)  
    D.train(ref4, torch.FloatTensor([1.0]))
    D.train(G.forward(generate_random_seed(latent_space)).detach(),torch.FloatTensor([0.0]))
    G.train(D, generate_random_seed(latent_space), torch.FloatTensor([1.0]))   
    '''
    
    pass
  pass
print("Iteration 1 is Finished")

'''
epochs2 = 100
iteration2 = int(len(train_gan)/Data_size)

for i in range(iteration2) : #i는 0부터 시작 
  print("Second Iteration : ", i)
  for epoch in range(epochs2) :
    ref = train_gan[(i) * Data_size : (i+1) * Data_size]
    #ref = train_gan[(i)*int(Data_size/2) : (i+2)*int(Data_size/2)] #Normalizing된 신호
    ref = torch.tensor(ref, dtype = torch.float32)

    D.train(ref, torch.FloatTensor([1.0]))
    D.train(G.forward(generate_random_seed(latent_space)).detach(),torch.FloatTensor([0.0]))

    G.train(D, generate_random_seed(latent_space), torch.FloatTensor([1.0]))
    pass
  pass
print("Iteration 2 is Finished")

'''

In [ ]:
seed = generate_random_seed(latent_space)
out = G.forward(seed)
out = out.detach().cpu().numpy()
print(out.shape) # [1,2,60]
out_dim = back_MinMax(out, arr_max1, arr_min1)
print(out_dim.shape)
plt.figure()
out_dim1_denorm = out_dim[0,0,:]
out_dim2_denorm = out_dim[0,1,:]
out_dim3_denorm = out_dim[0,2,:]
out_dim4_denorm = out_dim[0,3,:]

#Original
df_origin = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Batt/train_v.txt') 
train_gan_graph = df_origin.iloc[:time, -4:].values
print(train_gan_graph)

plt.plot(out_dim1_denorm, label = "train_gan - Temp")
plt.legend(prop = {'size':10})

plt.figure()
plt.plot(train_gan_graph[:,0], label = "Original - Temp")
plt.legend(prop = {'size':10})

plt.figure()
plt.plot(out_dim2_denorm, label = "train_gan - Capacity")
plt.legend(prop = {'size':10})

plt.figure()
plt.plot(train_gan_graph[:,1], label = "Original - Capacity")
plt.legend(prop = {'size':10})

plt.figure()
plt.plot(out_dim3_denorm, label = "train_gan - Current")
plt.legend(prop = {'size':10})

plt.figure()
plt.plot(train_gan_graph[:,2], label = "Original - Current")
plt.legend(prop = {'size':10})

plt.figure()
plt.plot(out_dim4_denorm, label = "train_gan - Voltage")
plt.legend(prop = {'size':10})

plt.figure()
plt.plot(train_gan_graph[:,3], label = "Original - Voltage")
plt.legend(prop = {'size':10})


In [ ]:
'''
#Test Training

iteration3 = 1
epochs3 = 1000

for i in range(iteration3) : #i는 0부터 시작 
  print("First Iteration : ",i)
  for epoch in range(epochs3) :
    #ref = train_gan[(i) * Data_size : (i+1) * Data_size]
    ref = train_gan[(i)*int(Data_size/2) : (i+2)*int(Data_size/2)] #Normalizing된 신호
    ref = torch.tensor(ref, dtype = torch.float32)

    D.train(ref, torch.FloatTensor([1.0]))
    D.train(G.forward(generate_random_seed(latent_space)).detach(),torch.FloatTensor([0.0]))

    G.train(D, generate_random_seed(latent_space), torch.FloatTensor([1.0]))
    pass
  pass

  print("Iteration 3 is Finished")
  
  '''

In [ ]:
#Plotting the Generator Loss
D.plot_progress()
G.plot_progress()
print(len(train_gan)) 

In [ ]:
# Dimension 1 Data , Voltage
seed1 = generate_random_seed(latent_space)
out1 = G.forward(seed)
out1 = out1.detach().cpu().numpy()
out_dim1 = back_MinMax(out1, arr_max1, arr_min1)
print(out_dim.shape)
plt.figure()
out_dim1_denorm = out_dim1[0,0,:]
out_dim2_denorm = out_dim1[0,1,:] #Voltage
out_dim3_denorm = out_dim1[0,2,:]

plt.figure()
plt.plot(out_dim2_denorm, label = "train_gan - Capacity 1")
plt.legend(prop = {'size':10})

seed1 = generate_random_seed(latent_space)
out1 = G.forward(seed)
out1 = out1.detach().cpu().numpy()
out_dim1 = back_MinMax(out1, arr_max1, arr_min1)
print(out_dim.shape)
plt.figure()
out_dim1_denorm = out_dim1[0,0,:]
out_dim2_denorm = out_dim1[0,1,:] #Voltage
out_dim3_denorm = out_dim1[0,2,:]

plt.figure()
plt.plot(out_dim2_denorm, label = "train_gan - Capacity 2")
plt.legend(prop = {'size':10})

seed1 = generate_random_seed(latent_space)
out1 = G.forward(seed)
out1 = out1.detach().cpu().numpy()
out_dim1 = back_MinMax(out1, arr_max1, arr_min1)
print(out_dim.shape)
plt.figure()
out_dim1_denorm = out_dim1[0,0,:]
out_dim2_denorm = out_dim1[0,1,:] #Voltage
out_dim3_denorm = out_dim1[0,2,:]

plt.figure()
plt.plot(out_dim2_denorm, label = "train_gan - Capacity 3")
plt.legend(prop = {'size':10})


plt.figure()
plt.plot(train_gan_graph[:,1], label = "Original - Capacity")
plt.legend(prop = {'size':10})



In [ ]:
# Dimension 2 Data 
seed1 = generate_random_seed(latent_space)
out1 = G.forward(seed)
out1 = out1.detach().cpu().numpy()
out_dim1 = back_MinMax(out1, arr_max1, arr_min1)
print(out_dim.shape)
plt.figure()
out_dim1_denorm = out_dim1[0,0,:]
out_dim2_denorm = out_dim1[0,1,:] #Voltage
out_dim3_denorm = out_dim1[0,2,:]

plt.figure()
plt.plot(out_dim1_denorm, label = "train_gan - Temp")
plt.legend(prop = {'size':10})

seed1 = generate_random_seed(latent_space)
out1 = G.forward(seed)
out1 = out1.detach().cpu().numpy()
out_dim1 = back_MinMax(out1, arr_max1, arr_min1)
print(out_dim.shape)
plt.figure()
out_dim1_denorm = out_dim1[0,0,:]
out_dim2_denorm = out_dim1[0,1,:] #Voltage
out_dim3_denorm = out_dim1[0,2,:]

plt.figure()
plt.plot(out_dim1_denorm, label = "train_gan - Temp")
plt.legend(prop = {'size':10})

seed1 = generate_random_seed(latent_space)
out1 = G.forward(seed)
out1 = out1.detach().cpu().numpy()
out_dim1 = back_MinMax(out1, arr_max1, arr_min1)
print(out_dim.shape)
plt.figure()
out_dim1_denorm = out_dim1[0,0,:]
out_dim2_denorm = out_dim1[0,1,:] #Voltage
out_dim3_denorm = out_dim1[0,2,:]

plt.figure()
plt.plot(out_dim1_denorm, label = "train_gan - Temp")
plt.legend(prop = {'size':10})


plt.figure()
plt.plot(train_gan_graph[:,0], label = "Original Temp")
plt.legend(prop = {'size':10})



In [ ]:
# Dimension 2 Data 
seed1 = generate_random_seed(latent_space)
out1 = G.forward(seed)
out1 = out1.detach().cpu().numpy()
out_dim1 = back_MinMax(out1, arr_max1, arr_min1)
print(out_dim.shape)
plt.figure()
out_dim1_denorm = out_dim1[0,0,:]
out_dim2_denorm = out_dim1[0,1,:] #Voltage
out_dim3_denorm = out_dim1[0,2,:]

plt.figure()
plt.plot(out_dim3_denorm, label = "train_gan - Current 1")
plt.legend(prop = {'size':10})

seed1 = generate_random_seed(latent_space)
out1 = G.forward(seed)
out1 = out1.detach().cpu().numpy()
out_dim1 = back_MinMax(out1, arr_max1, arr_min1)
print(out_dim.shape)
plt.figure()
out_dim1_denorm = out_dim1[0,0,:]
out_dim2_denorm = out_dim1[0,1,:] #Voltage
out_dim3_denorm = out_dim1[0,2,:]

plt.figure()
plt.plot(out_dim3_denorm, label = "train_gan - Current 2")
plt.legend(prop = {'size':10})

seed1 = generate_random_seed(latent_space)
out1 = G.forward(seed)
out1 = out1.detach().cpu().numpy()
out_dim1 = back_MinMax(out1, arr_max1, arr_min1)
print(out_dim.shape)
plt.figure()
out_dim1_denorm = out_dim1[0,0,:]
out_dim2_denorm = out_dim1[0,1,:] #Voltage
out_dim3_denorm = out_dim1[0,2,:]

plt.figure()
plt.plot(out_dim3_denorm, label = "train_gan - Current 3")
plt.legend(prop = {'size':10})

plt.figure()
plt.plot(train_gan_graph[:,2], label = "Original - Current")
plt.legend(prop = {'size':10})



In [ ]:
# Dimension 2 Data 
seed1 = generate_random_seed(latent_space)
out1 = G.forward(seed)
out1 = out1.detach().cpu().numpy()
out_dim1 = back_MinMax(out1, arr_max1, arr_min1)
print(out_dim.shape)
plt.figure()
out_dim1_denorm = out_dim1[0,0,:]
out_dim2_denorm = out_dim1[0,1,:] #Voltage
out_dim3_denorm = out_dim1[0,2,:]
out_dim4_denorm = out_dim1[0,3,:]

plt.figure()
plt.plot(out_dim4_denorm, label = "train_gan - Voltage 1")
plt.legend(prop = {'size':10})

seed1 = generate_random_seed(latent_space)
out1 = G.forward(seed)
out1 = out1.detach().cpu().numpy()
out_dim1 = back_MinMax(out1, arr_max1, arr_min1)
print(out_dim.shape)
plt.figure()
out_dim1_denorm = out_dim1[0,0,:]
out_dim2_denorm = out_dim1[0,1,:] #Voltage
out_dim3_denorm = out_dim1[0,2,:]
out_dim4_denorm = out_dim1[0,3,:]

plt.figure()
plt.plot(out_dim4_denorm, label = "train_gan - Voltage 2")
plt.legend(prop = {'size':10})

seed1 = generate_random_seed(latent_space)
out1 = G.forward(seed)
out1 = out1.detach().cpu().numpy()
out_dim1 = back_MinMax(out1, arr_max1, arr_min1)
print(out_dim.shape)
plt.figure()
out_dim1_denorm = out_dim1[0,0,:]
out_dim2_denorm = out_dim1[0,1,:] #Voltage
out_dim3_denorm = out_dim1[0,2,:]
out_dim4_denorm = out_dim1[0,3,:]

plt.figure()
plt.plot(out_dim4_denorm, label = "train_gan - Voltage 3")
plt.legend(prop = {'size':10})

plt.figure()
plt.plot(train_gan_graph[:,3], label = "Original - Voltage")
plt.legend(prop = {'size':10})



In [ ]:
t.toc()